In [ ]:
import openpyxl
import pandas as pd
import numpy as np
import pyodbc
import struct
import json

from os import name
from os.path import join
from openpyxl.styles import Font, Color, Alignment, PatternFill, Border, Side

from data.write_excel import (
    auto_dimension, add_result, result_style, table_record, create_month_report, 
    table_style, table_report_style, margin_res, gross_record)

In [ ]:
path = join('config', 'config.json')
with open(path, encoding='utf8') as fin:
    data = json.load(fin)
    dsn = data['dsn']
    uid = data['uid']
    pwd = data['pwd']
    base = data['base']
    driver = data['driver']
    server = data['server']
    database = data['database']

pd.options.display.max_columns = 100

In [ ]:
def handle_datetimeoffset(dto_value):
    # ref: https://github.com/mkleehammer/pyodbc/issues/134#issuecomment-281739794
    tup = struct.unpack("<6hI2h", dto_value) # e.g., (2017, 3, 16, 10, 35, 18, 0, -6, 0)
    tweaked = [tup[i] // 100 if i == 6 else tup[i] for i in range(len(tup))]
    return "{:04d}-{:02d}-{:02d} {:02d}:{:02d}:{:02d}.{:07d} {:+03d}:{:02d}".format(*tweaked)

In [ ]:
if name == 'nt':
    connect_db = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={uid};PWD={pwd}')
else:
    connect_db = pyodbc.connect(f'DSN={dsn};UID={uid};PWD={pwd}')
connect_db.add_output_converter(-155, handle_datetimeoffset)

# Даты

In [ ]:
today = pd.Timestamp.today()
# today = pd.Timestamp('2019-09-02')
today = pd.Timestamp(year=today.year, month=today.month, day=today.day)
month = (today - pd.Timedelta(days=3)).month
year = (today - pd.Timedelta(days=3)).year

months = ['', 'ЯНВАРЬ', 'ФЕВРАЛЬ', 'МАРТ', 
          'АПРЕЛЬ', 'МАЙ', 'ИЮНЬ',
          'ИЮЛЬ', 'АВГУСТ', 'СЕНТЯБРЬ',
          'ОКТЯБРЬ', 'НОЯБРЬ', 'ДЕКАБРЬ',]

weekdays = ['вс', 'пн', 'вт', 'ср', 'чт', 'пт', 'сб']

start_last_week = today + pd.Timedelta(days=-today.weekday(), weeks=-1)
end_last_week = start_last_week + pd.Timedelta(days=6)
last_day_for_fine_and_done = start_last_week + pd.Timedelta(days=11)

# Исходные таблицы

In [ ]:
sql = f'SELECT * FROM {base}.[BaseQuestionary];'
base_questionary = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[Questionarys];'
questionarys = pd.read_sql(sql, connect_db)

questionarys['Referrer'].fillna('', inplace=True)
questionarys['ShiftCount'].fillna(0, inplace=True)
questionarys['Vozrast'].fillna('', inplace=True)
questionarys['Nomerkarty'].fillna('', inplace=True)
questionarys['Vladeleckarty'].fillna('', inplace=True)
questionarys['Telefon'].fillna('', inplace=True)

In [ ]:
sql = f'SELECT * FROM {base}.[EmployeeStatus];'
employee_status = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[EmploymentStatus];'
employment_status = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[EmploymentType];'
employment_type = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[Specialty];'
specialty = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[Appearance];'
appearance = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[Citizenship];'
citizenship = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[InterstSource];'
interst_source = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[InterstSourceDetails];'
interst_source_details = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[PhoneType];'
phone_type = pd.read_sql(sql, connect_db)

sql = f'SELECT * FROM {base}.[QuestionaryPhones];'
questionary_phones = pd.read_sql(sql, connect_db)

questionary_phones['CommunicationType'] = [phone_type[phone_type['Id'] == x]['Name'].values.item(0)
                                           if pd.notna(x) else ''
                                           for x in questionary_phones['CommunicationType']]

In [ ]:
sql = f'SELECT * FROM {base}.[BaseFacility];'
base_facility = pd.read_sql(sql, connect_db)

def add_object(s):
    return f'Object{s}'
base_facility.columns = list(map(add_object, base_facility.columns))

In [ ]:
sql = f'SELECT * FROM {base}.[FineReason];'
fine_reason = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[BalanceType];'
balance_type = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[Shift];'
shift = pd.read_sql(sql, connect_db)

shift['IsShiftPaid'].fillna(False, inplace=True)
shift['FineOrBonus'].fillna(0, inplace=True)
shift['ResultOfShift'].fillna(0, inplace=True)

In [ ]:
sql = f'SELECT * FROM {base}.[Partners];'
partners = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[Contact];'
contact = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[BaseCity];'
base_city = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[RateForPartner];'
rate_for_partner = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[InteriorRates];'
interior_rates = pd.read_sql(sql, connect_db)

In [ ]:
sql = f'SELECT * FROM {base}.[AppointedStaff];'
appointed_staff = pd.read_sql(sql, connect_db)

# Данные по сотрудникам

In [ ]:
merge = ['Id', 'Referrer', 'Obekt', 'InterstSourceDetails', 'ShiftCount', 'Vozrast', 'Nomerkarty', 'Vladeleckarty', 'Telefon']

full_questionarys = base_questionary.merge(questionarys[merge], on='Id', how='left', validate='m:1')
# full_questionarys = full_questionarys.merge(questionary_phones, left_on='Id', right_on='Questionary', how='left', validate='m:1')
# full_questionarys = full_questionarys.merge(base_facility, left_on='Obekt', right_on='ObjectId', how='left', validate='m:1')

full_questionarys['EmployeeStatus'] = [employee_status[employee_status['Id'] == x]['Name'].values.item(0)
                                       if pd.notna(x) else ''
                                       for x in full_questionarys['EmployeeStatus']]
full_questionarys['EmploymentStatus'] = [employment_status[employment_status['Id'] == x]['Name'].values.item(0)
                                         if pd.notna(x) else ''
                                         for x in full_questionarys['EmploymentStatus']]
full_questionarys['EmploymentType'] = [employment_type[employment_type['Id'] == x]['Name'].values.item(0)
                                       if pd.notna(x) else ''
                                       for x in full_questionarys['EmploymentType']]
full_questionarys['Speciality'] = [specialty[specialty['Id'] == x]['Name'].values.item(0)
                                   if pd.notna(x) else ''
                                   for x in full_questionarys['Speciality']]
full_questionarys['Appearance'] = [appearance[appearance['Id'] == x]['Name'].values.item(0)
                                   if pd.notna(x) else ''
                                   for x in full_questionarys['Appearance']]
full_questionarys['Citizenship'] = [citizenship[citizenship['Id'] == x]['Name'].values.item(0)
                                   if pd.notna(x) else ''
                                   for x in full_questionarys['Citizenship']]
full_questionarys['InterstSource'] = [interst_source[interst_source['Id'] == x]['Name'].values.item(0)
                                   if pd.notna(x) else ''
                                   for x in full_questionarys['InterstSource']]
full_questionarys['InterstSourceDetails'] = [interst_source_details[interst_source_details['Id'] == x]['Name'].values.item(0)
                                             if pd.notna(x) else ''
                                             for x in full_questionarys['InterstSourceDetails']]

drop = ['PassportIssuedBy', 'Specialization', 'FactAddress', 'MidicalCardExpireDate', 'WatchValidity', 'DistantWorkCompany',
       'Address', 'IsMedicalCard', 'ShiftsCount', 'DistantWorkExperience', 'HRmanager', 'Gender', 'Obekt', 
       'CustObjVersion', 'IdCreateBy', 'CreateDate', 'IdUpdateBy', 'UpdateDate']

full_questionarys.drop(drop, axis=1, inplace=True)

In [ ]:
full_questionarys[full_questionarys['Id'] == 184130]

# Данные по оплатам

In [ ]:
def add_price(x):
    partner, specialty = x
    if pd.notna(partner) and pd.notna(specialty):
        return rate_for_partner[(rate_for_partner['Klient'] == partner) & 
                                (rate_for_partner['Specialty'] == specialty)]['Price'].values.item(0)
    else:
        return 0

shift_result = shift.merge(base_facility, left_on='Facility', right_on='ObjectId', how='left', validate='m:1')

shift_result['FineReason'] = [fine_reason[fine_reason['Id'] == x]['Name'].values.item(0)
                              if pd.notna(x) else ''
                              for x in shift_result['FineReason']]

shift_result = shift_result.rename(columns={'Id_x': 'Id'})
shift_result.index = shift_result['Id']

drop = ['ObjectTaskMaster', 'IsAuto', 'Comment', 'Id', 
        'ObjectCustObjVersion', 'ObjectIdCreateBy', 'ObjectCreateDate', 'ObjectIdUpdateBy', 'ObjectUpdateDate']
shift_result.drop(drop, axis=1, inplace=True)

shift_result['Date'] = [pd.Timestamp(date[:10]) for date in shift_result['Date']]
shift_result['PaymentDate'] = [pd.Timestamp(date[:10]) 
                               if date else pd.Timestamp('1900-01-01')
                               for date in shift_result['PaymentDate']]

shift_result['PartnerName'] = [partners[partners['Id'] == x]['LegalName'].values.item(0)
                               if pd.notna(x) else 'Владелец'
                               for x in shift_result['Partner']]

shift_result['WhoPaidMoney'] = [contact[contact['Id'] == x]['Partner'].values.item(0)
                                if pd.notna(x) else ''
                                for x in shift_result['WhoPaidMoney']]

merge = ['Id', 'InnerRate', 'Specialty', 'Vydanonalichnymi']
shift_result = shift_result.merge(appointed_staff[merge], left_on='AppointedStaff', right_on='Id', how='left', validate='m:1')
shift_result['Vydanonalichnymi'].fillna(0, inplace=True)
for index, row in shift_result[(shift_result['Vydanonalichnymi'] != 0)].iterrows():
    row['WorkedHours'] = 0
    row['FineOrBonus'] = 0
    row['IsShiftPaid'] = False
    row['IsComeToWork'] = None
    row['PaymentDate'] = row['Date']
    row['Type'] = 127
    row['WhoPaidMoney'] = row['Partner']
    shift_result = shift_result.append(row, ignore_index=True)

shift_result['InnerRate_y'] = [interior_rates[interior_rates['Id'] == x]['LocalWorkPrice'].values.item(0)
                               if pd.notna(x) else 0
                               for x in shift_result['InnerRate_y']]

shift_result = shift_result.assign(PriceForHour=shift_result[['Partner', 'Specialty']].apply(add_price, axis=1))
shift_result['WorkedHours'].fillna(0, inplace=True)
shift_result['InnerRate_total'] = [shift_result['InnerRate_x'][i]
                                   if pd.notna(shift_result['InnerRate_x'][i]) else shift_result['InnerRate_y'][i]
                                   for i, _ in enumerate(shift_result['InnerRate_y'])]
shift_result['ResultOfShift_total'] = [shift_result['WorkedHours'][i] * shift_result['InnerRate_total'][i] 
                                       if shift_result['ResultOfShift'][i] == 0 
                                       else shift_result['ResultOfShift'][i]
                                       for i, _ in enumerate(shift_result['WorkedHours'])]
shift_result['ResultOfShift_total'] = [cash
                                       if shift_result['ResultOfShift_total'][i] == 0 
                                       else shift_result['ResultOfShift_total'][i]
                                       for i, cash in enumerate(shift_result['Vydanonalichnymi'])]
shift_result['Price_total'] = shift_result['WorkedHours'] * shift_result['PriceForHour']
shift_result['Margin'] = shift_result['Price_total'] - shift_result['ResultOfShift_total']
shift_result['month'] = [date.month for date in shift_result['Date']]

drop = ['AppointedStaff', 'ObjectSheduleFrom', 'ObjectSheduleTo', 'ObjectWayDescription', 'Id', 
       'CustObjVersion', 'IdCreateBy', 'CreateDate', 'IdUpdateBy', 'UpdateDate']
shift_result.drop(drop, axis=1, inplace=True)

In [ ]:
shift_result[(shift_result['Questionary'] == 184172)]

# ЗП ведомость:  
* столбец выдано - это уже выдано наличными  
* столбец долг - это остаток нашей задолженности перед работниками после вычета штрафов и уже выданных денег наличными.  

В ведомость надо добавить:  
* столбец с информацией на каком объекте был штраф и дата штрафа  
* Тоже с выданными наличными, на каком объекте и дата выдачи наличных.   
* Штрафы в ведомости отображаются не только за прошедшую неделю, но и за текущую до дня выдачи денег (пятница). Например работник отработал прошлую неделю без штрафов, в пн на этой неделе начислено 5000р к выдаче, но во вт он не вышел, значит этот штраф с текущей недели мы учитываем в ведомости за прошлую неделю.  

Ведомость составляется еженедельно в понедельник за прошедшую неделю

## Таблицы с ведомостями

In [ ]:
group_by = ['Questionary']
col_shift = group_by + ['ResultOfShift_total']

group_by_fine = group_by
col_fine = group_by_fine + ['FineOrBonus']

group_by_fine_detail = group_by + ['ObjectName', 'Date', 'FineReason']
col_fine_detail = group_by_fine_detail + ['FineOrBonus']

group_by_done = group_by
col_done = group_by_done + ['ResultOfShift_total']

group_by_done_detail = group_by + ['WhoPaidMoney', 'PaymentDate']
col_done_detail = group_by_done_detail + ['ResultOfShift_total']

salary_fine = shift_result[col_fine][(shift_result['Type'] == 126) & 
                                     (shift_result['Date'] >= start_last_week) & 
                                     (shift_result['Date'] <= last_day_for_fine_and_done)]
salary_fine = salary_fine.groupby(group_by_fine).agg('sum').reset_index()
salary_fine = salary_fine.rename(columns={'FineOrBonus': 'Штраф'})

salary_done = shift_result[col_done][(shift_result['Type'] != 126) & 
                                     (shift_result['IsShiftPaid'] == False) &
                                     (shift_result['Date'] >= start_last_week) & 
                                     (shift_result['Date'] <= last_day_for_fine_and_done)]
salary_done = salary_done.groupby(group_by_done).agg('sum').reset_index()
salary_done = salary_done.rename(columns={'ResultOfShift_total': 'Выплачено'})

salary_record = shift_result[col_shift][(shift_result['Type'] == 126) & 
                                        (shift_result['Date'] >= start_last_week) & 
                                        (shift_result['Date'] <= end_last_week)]
salary_record = salary_record.groupby(group_by).agg('sum').reset_index()
salary_record = salary_record.rename(columns={'ResultOfShift_total': 'Начислено'})
salary_record = salary_record.merge(salary_done, on='Questionary', how='outer', validate='1:1')
salary_record = salary_record.merge(salary_fine, on='Questionary', how='outer', validate='1:1')

salary_fine_detail = shift_result[col_fine_detail][(shift_result['FineOrBonus'] != 0) & 
                                                   (shift_result['Date'] >= start_last_week) & 
                                                   (shift_result['Date'] <= last_day_for_fine_and_done)]
salary_fine_detail = salary_fine_detail.groupby(group_by_fine_detail).agg('sum').reset_index()
salary_fine_detail = salary_fine_detail.rename(columns={'FineOrBonus': 'Штраф', 'ObjectName': 'На каком объекте был штраф', 
                                                        'Date': 'Дата штрафа', 'FineReason': 'Причина штрафа'})

salary_done_detail = shift_result[col_done_detail][(shift_result['Type'] != 126) & 
                                                   (shift_result['IsShiftPaid'] == False) &
                                                   (shift_result['Date'] >= start_last_week) &
                                                   (shift_result['Date'] <= last_day_for_fine_and_done)]
salary_done_detail['WhoPaidMoney'] = [partners[partners['Id'] == x]['LegalName'].values.item(0)
                                      if pd.notna(x) else 'Зарплата'
                                      for x in salary_done_detail['WhoPaidMoney']]
salary_done_detail = salary_done_detail.groupby(group_by_done_detail).agg('sum').reset_index()
salary_done_detail = salary_done_detail.rename(columns={'ResultOfShift_total': 'Выплачено', 'WhoPaidMoney': 'Кем выплачено', 
                                                        'PaymentDate': 'Дата выплаты'})

salary_record.fillna(0, inplace=True)
salary_fine_detail.fillna(0, inplace=True)
salary_done_detail.fillna(0, inplace=True)

merge_record = ['Id', 'FIO', 'Nomerkarty', 'Vladeleckarty']
merge_other = ['Id', 'FIO']
salary_record = salary_record.merge(full_questionarys[merge_record], left_on='Questionary', right_on='Id', how='left', validate='m:1')
salary_fine_detail = salary_fine_detail.merge(full_questionarys[merge_other], left_on='Questionary', right_on='Id', 
                                              how='left', validate='m:1')
salary_done_detail = salary_done_detail.merge(full_questionarys[merge_other], left_on='Questionary', right_on='Id', 
                                              how='left', validate='m:1')

drop = ['Questionary', 'Id']
salary_record.drop(drop, axis=1, inplace=True)
salary_fine_detail.drop(drop, axis=1, inplace=True)
salary_done_detail.drop(drop, axis=1, inplace=True)

In [ ]:
def duty(x):
    shift, fine, done = x
    if shift > done - fine:
        return shift - done + fine
    else:
        return 0
    
salary_record = salary_record.assign(duty=salary_record[['Начислено', 'Штраф', 'Выплачено']].apply(duty, axis=1))
salary_record = salary_record.rename(columns={'FIO': 'ФИО', 'Nomerkarty': '№ карты', 'Vladeleckarty': 'Владелец карты', 
                                              'duty': 'Долг'})
salary_fine_detail = salary_fine_detail.rename(columns={'FIO': 'ФИО'})
salary_done_detail = salary_done_detail.rename(columns={'FIO': 'ФИО'})

columns_record = ['ФИО', '№ карты', 'Владелец карты', 'Начислено', 'Штраф', 'Выплачено', 'Долг']
columns_fine_detail = ['ФИО', 'Причина штрафа', 'На каком объекте был штраф', 'Дата штрафа', 'Штраф']
columns_done_detail = ['ФИО','Кем выплачено', 'Дата выплаты', 'Выплачено']

salary_record = salary_record[columns_record]
salary_record.sort_values(by=['Долг'], ascending=False, inplace=True)
salary_fine_detail = salary_fine_detail[columns_fine_detail]
salary_done_detail = salary_done_detail[columns_done_detail]

In [ ]:
salary_done_detail

# Запись в Excel

In [ ]:
file_name = f'{today:%Y-%m-%d}. salary.xlsx'
sheet_name = 'salary_record'
font_title = Font(bold=True, size=14, color='0C4561')

In [ ]:
work_book = openpyxl.Workbook()
sheet = work_book.active
sheet.title = sheet_name

current_row = 1
sheet.cell(row=current_row, column=1).value = f'ВЕДОМОСТЬ ПО ЗП'
sheet.cell(row=current_row, column=1).font = font_title
current_row += 1

sheet.cell(row=current_row, column=1).value = f'Дата формирования отчета: '
sheet.cell(row=current_row, column=2).value = f'{today:%d.%m.%Y}'
current_row += 1

sheet.cell(row=current_row, column=1).value = f'Отчетные даты: '
sheet.cell(row=current_row, column=2).value = f'{start_last_week:%d.%m.%Y} - {end_last_week:%d.%m.%Y}'
current_row += 1

sheet.cell(row=current_row, column=1).value = f'Даты учитываемых штрафов и выплат: '
sheet.cell(row=current_row, column=2).value = f'{start_last_week:%d.%m.%Y} - {last_day_for_fine_and_done:%d.%m.%Y}'
current_row += 1

work_book.save(file_name)

In [ ]:
work_book = openpyxl.load_workbook(file_name)
sheet = work_book[sheet_name]

with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', datetime_format='DD.MM.YYYY') as writer:
    writer.book = work_book
    writer.sheets = dict((ws.title, ws) for ws in work_book.worksheets)

    res = ((1, 'ИТОГО'),
           (4, sum(salary_record['Начислено'])),
           (5, sum(salary_record['Штраф'])),
           (6, sum(salary_record['Выплачено'])),
           (7, sum(salary_record['Долг'])))
    current_row = table_record(writer, sheet, salary_record, res, row=current_row, is_auto_dimension=True)

    if salary_fine_detail.shape[0] != 0:
        res = ((1, 'ИТОГО'),
               (5, sum(salary_fine_detail['Штраф'])))
        current_row = table_record(writer, sheet, salary_fine_detail, res, table_name='ШТРАФЫ', row=current_row)
        
    if salary_done_detail.shape[0] != 0:
        res = ((1, 'ИТОГО'),
               (4, sum(salary_done_detail['Выплачено'])))
        current_row = table_record(writer, sheet, salary_done_detail, res, table_name='ВЫПЛАТЫ', row=current_row)

work_book.save(file_name)

# Табель

In [ ]:
merge = ['Id', 'FIO']
shift_for_report = shift_result.merge(full_questionarys[merge], left_on='Questionary', right_on='Id', how='left', validate='m:1')

drop = ['FineOrBonus', 'ResultOfShift', 'IsShiftPaid', 'PaymentDate', 'IsComeToWork', 'InnerRate_x', 'InstanceDate', 
        'WhoPaidMoney', 'FineReason', 'Id', 'Questionary', 'ObjectPartner', 'InnerRate_y', 'Specialty', 'PriceForHour', 
        'InnerRate_total']
shift_for_report.drop(drop, axis=1, inplace=True)

shift_for_report.sort_values(by=['FIO'], inplace=True)
shift_for_report = shift_for_report[shift_for_report['month'] == month]

done_for_report = shift_for_report[shift_for_report['Type'] != 126]
shift_for_report = shift_for_report[shift_for_report['Type'] == 126]

In [ ]:
done_for_report.head()

In [ ]:
group_by = ['Partner', 'PartnerName', 'ObjectName', 'ObjectAddress']
shift_for_report_unique = pd.DataFrame({'WorkedHours' : shift_for_report.groupby(group_by)['WorkedHours'].sum()}).reset_index()
shift_for_report_unique

## Запись в Excel

In [ ]:
sheet_name = 'shift_report'
work_book = openpyxl.load_workbook(file_name)

with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', datetime_format='DD/MM') as writer:
    writer.book = work_book
    
    for obj in shift_for_report_unique['ObjectName']:
        month_report_obj = create_month_report(shift_for_report, obj)
        
        partner_id = int(shift_for_report_unique[shift_for_report_unique['ObjectName'] == obj]['Partner'])
        partner = shift_for_report_unique[shift_for_report_unique['ObjectName'] == obj]['PartnerName'].values[0]
        address = shift_for_report_unique[shift_for_report_unique['ObjectName'] == obj]['ObjectAddress'].values[0]
        cur_sheet_name = f'{sheet_name}_{partner_id}'
        
        work_book.create_sheet(cur_sheet_name)
        sheet = work_book[cur_sheet_name]
        writer.sheets = dict((ws.title, ws) for ws in work_book.worksheets)
        
        row = 1
        sheet.cell(row=row, column=1).value = f'Лист учета рабочего времени {months[month]} {year}'
        sheet.cell(row=row, column=1).font = font_title
        row += 1
        sheet.cell(row=row, column=1).value = f'Дата формирования отчета: '
        sheet.cell(row=row, column=2).value = f'{today:%d.%m.%Y}'
        row += 1
        sheet.cell(row=row, column=1).value = 'ПАРТНЕР'
        sheet.cell(row=row, column=1).font = font_title
        sheet.cell(row=row, column=2).value = partner
        row += 1
        sheet.cell(row=row, column=1).value = 'ОБЪЕКТ'
        sheet.cell(row=row, column=1).font = font_title
        sheet.cell(row=row, column=2).value = obj
        row += 1
        sheet.cell(row=row, column=1).value = 'АДРЕС ОБЪЕКТА'
        sheet.cell(row=row, column=1).font = font_title
        sheet.cell(row=row, column=2).value = address
        row += 1
        
        month_report_obj.to_excel(writer, cur_sheet_name, startrow=row)
        row += 1
        sheet.cell(row=row, column=1).value = 'ФИО сотрудника'
        
        sheet.insert_rows(row + 1)
        for i, cell in enumerate(sheet[row + 1]):
            if i < 2:
                continue
            weekday = int(f'{sheet.cell(row=row, column=i + 1).value:%w}')
            weekday = weekdays[weekday]
            cell.value = weekday
        
        for i, _ in enumerate(sheet):
            sheet.cell(row=i + 1, column=1).alignment = Alignment(horizontal='left')
            
        res = [(1, 'ИТОГО ЧАСОВ')] + list((i + 2, sum(month_report_obj[column])) for i, column in enumerate(month_report_obj.columns))
        add_result(sheet, row + month_report_obj.shape[0] + 2, *res)
        
        for i, cell in enumerate(sheet[row]):
            if i < 2:
                continue
            cell.value = f'{cell.value:%d.%m}'
        auto_dimension(sheet, n_row=row - 1)
        
        table_report_style(sheet, row, row + month_report_obj.shape[0] + 2, month_report_obj.shape[1] + 1, 
                           formatting=True, is_result_style=[True, True])
        
        margin = shift_for_report[shift_for_report['ObjectName'] == obj][['Date', 
                                                                          'Price_total']].groupby(['Date']).agg('sum').reset_index()
        done = done_for_report[done_for_report['ObjectName'] == obj][['Date', 
                                                                      'ResultOfShift_total']].groupby(['Date']).agg('sum').reset_index()
        res_margin = [(1, 'ИТОГО ВЫРУЧКА'), (2, sum(margin['Price_total']))]
        res_done = [(1, 'ВЫПЛАЧЕНО ЗАКАЗЧИКОМ'), (2, sum(done['ResultOfShift_total']))]
        for i, date in enumerate(month_report_obj.columns[1:]):
            if date in list(margin['Date']):
                res_margin += [(i + 3, margin[margin['Date'] == date]['Price_total'].values[0])]
            else:
                res_margin += [(i + 3, 0)]
            if date in list(done['Date']):
                res_done += [(i + 3, done[done['Date'] == date]['ResultOfShift_total'].values[0])]
            else:
                res_done += [(i + 3, 0)]
        
        add_result(sheet, row + month_report_obj.shape[0] + 3, *res_margin)
        result_style(sheet[row + month_report_obj.shape[0] + 3], month_report_obj.shape[1] + 1, last_res=True)
        add_result(sheet, row + month_report_obj.shape[0] + 4, *res_done)
        result_style(sheet[row + month_report_obj.shape[0] + 4], month_report_obj.shape[1] + 1, last_res=True)
        
work_book.save(file_name)

# Валовая прибыль
### Прошлая неделя

In [ ]:
group_by = ['PartnerName', 'Partner']
column = group_by + ['ResultOfShift_total', 'Price_total', 'Margin', 'Vydanonalichnymi']

gross_margin = shift_result[(shift_result['Type'] == 126) & 
                            (shift_result['Date'] >= start_last_week) & 
                            (shift_result['Date'] <= end_last_week)]
gross_margin = gross_margin[column].groupby(group_by).agg('sum').reset_index()

gross_margin['City'] = [partners[partners['Id'] == x]['Gorod'].values.item(0)
                        if pd.notna(x) else ''
                        for x in gross_margin['Partner']]
gross_margin['City'] = [base_city[base_city['Id'] == x]['Name'].values.item(0)
                        if pd.notna(x) else ''
                        for x in gross_margin['City']]

drop = ['Partner']
gross_margin.drop(drop, axis=1, inplace=True)

gross_margin.sort_values(by=['Margin'], ascending=False, inplace=True)
gross_margin = gross_margin.rename(columns={'PartnerName': 'Заказчик', 'ResultOfShift_total': 'Себестоимость', 
                                            'Price_total': 'Выручка', 'Margin': 'Валовая прибыль', 'Vydanonalichnymi': 'Выплачено', 
                                            'City': 'Город'})
column = ['Город', 'Заказчик', 'Себестоимость', 'Выручка', 'Валовая прибыль', 'Выплачено']
gross_margin = gross_margin[column]
gross_margin

### Месяц

In [ ]:
group_by = ['PartnerName', 'Partner']
column = group_by + ['ResultOfShift_total', 'Price_total', 'Margin', 'Vydanonalichnymi']

gross_margin_month = shift_result[(shift_result['Type'] == 126) & 
                            (shift_result['month'] == month)]
gross_margin_month = gross_margin_month[column].groupby(group_by).agg('sum').reset_index()

gross_margin_month['City'] = [partners[partners['Id'] == x]['Gorod'].values.item(0)
                              if pd.notna(x) else ''
                              for x in gross_margin_month['Partner']]
gross_margin_month['City'] = [base_city[base_city['Id'] == x]['Name'].values.item(0)
                              if pd.notna(x) else ''
                              for x in gross_margin_month['City']]

drop = ['Partner']
gross_margin_month.drop(drop, axis=1, inplace=True)

gross_margin_month.sort_values(by=['Margin'], ascending=False, inplace=True)
gross_margin_month = gross_margin_month.rename(columns={'PartnerName': 'Заказчик', 'ResultOfShift_total': 'Себестоимость', 
                                                        'Price_total': 'Выручка', 'Margin': 'Валовая прибыль',
                                                        'Vydanonalichnymi': 'Выплачено', 'City': 'Город'})
column = ['Город', 'Заказчик', 'Себестоимость', 'Выручка', 'Валовая прибыль', 'Выплачено']
gross_margin_month = gross_margin_month[column]
gross_margin_month

## Записть в Excel

In [ ]:
file_name = f'{today:%Y-%m-%d}. margin.xlsx'
sheet_name = 'gross_margin_month'

work_book = openpyxl.Workbook()
sheet = work_book.active
sheet.title = sheet_name

work_book.save(file_name)

In [ ]:
work_book = openpyxl.load_workbook(file_name)
sheet = work_book[sheet_name]

row = 1
row = gross_record(gross_margin_month, sheet, file_name, work_book, is_month=True)

sheet_name = 'gross_margin'
work_book.create_sheet(sheet_name)
sheet = work_book[sheet_name]

row = gross_record(gross_margin, sheet, file_name, work_book)
        
work_book.save(file_name)

In [ ]:
print(f'Скрипт закончил работу. Сформирован файл "{file_name}"')